In [1]:
#%%
%reload_ext autoreload
%autoreload 2


In [1]:
import numpy as np
from metrics import calculate_sample_alignment_distance, calculate_sample_alignment_accuracy, calculate_class_alignment_distance, calculate_class_alignment_consistency, calculate_gaussian_potential_uniformity, calculate_probabilistic_entropy_uniformity, pairwise_distances
def gen_all_metrics(embeddings, labels):
    n_samples = embeddings.shape[0]
    all_embeddings = np.concatenate(embeddings, axis=0)
    all_labels = np.concatenate(labels, axis=0)


    # all_labels = labels
    # all_embeddings = embeddings
    #

    print(all_embeddings.shape, all_labels.shape)

    indices_0 = np.where(all_labels == 0)[0]
    indices_1 = np.where(all_labels == 1)[0]

    # Simulate similarity matrix
    sim = pairwise_distances(
        all_embeddings, metric="euclidean") + np.eye(all_embeddings.shape[0])

    sad_0, sad_1 = calculate_sample_alignment_distance(
        sim, n_samples, labels[0])

    saa_0, saa_1 = calculate_sample_alignment_accuracy(
        sim, n_samples, labels[0],all_labels)

    cad_0, cad_1 = calculate_class_alignment_distance(
        sim, all_embeddings, all_labels)

    cac_0, cac_1 = calculate_class_alignment_consistency(
        sim, all_embeddings, all_labels)

    # Gaussian Potential Uniformity
    gu_0, gu_1 = calculate_gaussian_potential_uniformity(
        all_embeddings, all_labels)

    # Probabilistic Entropy Uniformity
  

 

    print(f"{sad_0.mean():.3f} & {sad_1.mean():.3f} &", f"{(saa_0 / indices_0.shape[0]):.1f} & {(saa_1 / indices_1.shape[0]):.1f}",
          f" & {np.array(cad_0).mean():.3f} & {np.array(cad_1).mean():.3f} & ",   f"{cac_0.mean():.1f} & {cac_1.mean():.1f}", f" & {gu_0:.3f} & {gu_1:.3f} &",
          )

    # Compile results
    results = {
        "sample_alignment_distance_cls0": sad_0,
        "sample_alignment_distance_cls1": sad_1,
        "sample_alignment_accuracy_cls0": saa_0,
        "sample_alignment_accuracy_cls1": saa_1,
        "class_alignment_distance_cls0": cad_0,
        "class_alignment_distance_cls1": cad_1,
        "class_alignment_consistency_cls0": cac_0,
        "class_alignment_consistency_cls1": cac_1,
        "gaussian_potential_uniformity_cls0": gu_0,
        "gaussian_potential_uniformity_cls1": gu_1,


    }

    return results

In [2]:
models = ['supcon','paco','KCLK3','TSCK3', 'SBC','BCL','SupMin','Prot']
res = {}

In [4]:
import pickle
import numpy as np
import random
class_name = 'infarction'
re = False

# random indices 


for model in models:

    with open(f'/vol/miltank/users/mildenbd/Projects/icml/embeddings_baselines_card/{class_name}/embeddings_5_last_train__{model}.pkl', 'rb') as file:
        embeddings = pickle.load(file)
        print(len(embeddings))
    with open(f'/vol/miltank/users/mildenbd/Projects/icml/embeddings_baselines_card/{class_name}/labels_5_last_train__{model}.pkl', 'rb') as file:
        labels = pickle.load(file)

    print(len(labels))
    # labels = np.array(labels)
    # embeddings = np.array(embeddings)
    # embeddings = embeddings.reshape(embeddings.shape[1],-1)
    # labels = labels.reshape(-1)
    # print(labels.shape)
    # print(embeddings.shape)
    # positive_indices = np.where(labels == 1)[0]
    # num_positives = len(positive_indices)
    # print(f"Number of positive samples (label=1): {num_positives}")
    # negative_indices = np.where(labels == 0)[0]
    # num_negatives = len(negative_indices)
    # print(f"Number of negative samples (label=0): {num_negatives}")
    
  
    # if num_negatives < num_positives:
    #     print("Not enough negative samples to balance the dataset.")
    #     continue  # Skip to the next model
  
    # sampled_neg_indices = np.random.choice(
    #     negative_indices,
    #     size=num_positives,
    #     replace=False
    # )
    # print(f"Sampled {len(sampled_neg_indices)} negative indices.")
    # selected_indices = np.concatenate((positive_indices, sampled_neg_indices))
    
   
    # np.random.shuffle(selected_indices)
    # print("Shuffled the selected indices.")
    

    # subset_labels = labels[selected_indices].tolist()
    # subset_embeddings = embeddings[selected_indices].tolist()

    #print(f"Subset labels: {len(labels[selected_indices])}")

    #print(f'{class_name}_{model}')
    res[f'{class_name}_{model}'] = gen_all_metrics(embeddings[selected_indices], labels[selected_indices])



5
5


NameError: name 'selected_indices' is not defined

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler


performance = {
    'supcon':61.9
    ,'paco':66.6
    ,'KCLK3':75.3
    ,'TSCK3':75.7,
     'SBC':70
     ,'BCL':73.4
     ,'SupMin':77.7
     ,'Prot':77.9
}

for model, perf in performance.items():
    res[model]['performance'] = perf


df = pd.DataFrame.from_dict(performance, orient='index').reset_index()
df = df.rename(columns={'index': 'model'})


metrics = [col for col in df.columns if col not in ['model', 'performance']]


sns.set(style="whitegrid")

# Determine the number of plots
num_metrics = len(metrics)
cols = 3  # Number of columns in the grid
rows = num_metrics // cols + int(num_metrics % cols > 0)

# Initialize the matplotlib figure
plt.figure(figsize=(cols * 5, rows * 4))

for idx, metric in enumerate(metrics, 1):
    plt.subplot(rows, cols, idx)
    sns.scatterplot(data=df, x=metric, y='performance', s=100)

    # Annotate each point with the model name
    for i in range(df.shape[0]):
        plt.text(x=df[metric][i]+0.001, y=df['performance'][i]+0.001, s=df['model'][i],
                 fontdict=dict(color='black', size=9),
                 horizontalalignment='left', verticalalignment='bottom')

    plt.title(f'Performance vs {metric}')
    plt.xlabel(metric)
    plt.ylabel('Performance')
    plt.tight_layout()

plt.suptitle('Model Performance vs Various Metrics', fontsize=16, y=1.02)
plt.show()

# Optional: Adding Regression Line and Correlation Coefficient
plt.figure(figsize=(cols * 5, rows * 4))

for idx, metric in enumerate(metrics, 1):
    plt.subplot(rows, cols, idx)
    sns.regplot(data=df, x=metric, y='performance', scatter=True, ci=None, line_kws={'color': 'red'})

    
    corr, _ = pearsonr(df[metric], df['performance'])
    plt.text(0.05, 0.95, f'r={corr:.2f}', transform=plt.gca().transAxes,
             horizontalalignment='left', verticalalignment='top',
             bbox=dict(boxstyle='round,pad=0.2', fc='yellow', alpha=0.3))

    
    for i in range(df.shape[0]):
        plt.text(x=df[metric][i]+0.001, y=df['performance'][i]+0.001, s=df['model'][i],
                 fontdict=dict(color='black', size=9),
                 horizontalalignment='left', verticalalignment='bottom')

    plt.title(f'Performance vs {metric}')
    plt.xlabel(metric)
    plt.ylabel('Performance')
    plt.tight_layout()

plt.suptitle('Model Performance vs Various Metrics with Correlation', fontsize=16, y=1.02)
plt.show()

# Optional: Normalizing Metrics
scaler = MinMaxScaler()
df_normalized = df.copy()
df_normalized[metrics] = scaler.fit_transform(df[metrics])

# Plotting Normalized Metrics
plt.figure(figsize=(cols * 5, rows * 4))

for idx, metric in enumerate(metrics, 1):
    plt.subplot(rows, cols, idx)
    sns.scatterplot(data=df_normalized, x=metric, y='performance', s=100)

    # Annotate each point with the model name
    for i in range(df_normalized.shape[0]):
        plt.text(x=df_normalized[metric][i]+0.01, y=df_normalized['performance'][i]+0.01, s=df_normalized['model'][i],
                 fontdict=dict(color='black', size=9),
                 horizontalalignment='left', verticalalignment='bottom')

    plt.title(f'Performance vs {metric} (Normalized)')
    plt.xlabel(f'{metric} (Normalized)')
    plt.ylabel('Performance')
    plt.tight_layout()

plt.suptitle('Model Performance vs Various Metrics (Normalized)', fontsize=16, y=1.02)
plt.show()


ModuleNotFoundError: No module named 'seaborn'